In [ ]:
# packages
import sys
import dask
import intake
import numpy as np
import pandas as pd
import xarray as xr

# link to self-written packages
sys.path.append("/g/data/if69/cj0591/repos/for_hackathon")
from utils.tools import write_to_filelist, clear_dir
from tempestextremes_utils.node_utils import run_detectNodes, run_stitchNodes

In [ ]:
# set dask workers
client = dask.distributed.Client(n_workers=28)

In [ ]:
# era5 category
cat = intake.cat.nci
era5 = cat['era5']

In [7]:
year_sta = 1979
year_end = 2020

In [ ]:
# To detect tropical cyclones, five variables are requred: topography (ZS),
# mean sea level pressure (msl), 10-m U wind (10u) and V wind (10-v), and geopotential (z).

In [ ]:
# topography is calculated from surface geopotential
zsfile = '/g/data/if69/cj0591/repos/tc-detection/data/zs_era5_oper_sfc_invariant.nc'

# geopotential
sub_z = era5.search(sub_collection='era5',product_type='reanalysis', year=list(range(year_sta ,year_end+1)),
                    dataset='pressure-levels', parameter='z')
z_files = sub_z.df.path.unique()

# mean sea level pressure
sub_msl = era5.search(sub_collection='era5',product_type='reanalysis', year=list(range(year_sta ,year_end+1)),
                      dataset='single-levels', parameter='msl')
msl_files = sub_msl.df.path.unique()

# 10-m u wind
sub_10u = era5.search(sub_collection='era5',product_type='reanalysis', year=list(range(year_sta ,year_end+1)),
                      dataset='single-levels', parameter='10u')
u10_files = sub_10u.df.path.unique()

# 10-m v wind
sub_10v = era5.search(sub_collection='era5',product_type='reanalysis', year=list(range(year_sta ,year_end+1)),
                      dataset='single-levels', parameter='10v')
v10_files = sub_10v.df.path.unique()

In [ ]:
## set directories (create directories accordingly)

# base directory
base_dir = '/g/data/if69/cj0591/repos/tc-detection'

# input & output directory
txt_dir = f'{base_dir}/txt' 
log_dir = f'{base_dir}/log' # log files
input_dir = f'{base_dir}/data/in' # input filelist
output_dir = f'{base_dir}/data/out' # output filelist
output_temp_dir = f'{base_dir}/data/temp' # temporary for output files

In [ ]:
# be very careful with this - it will delete everything in the directory!!!
clear_dir(log_dir)
clear_dir(input_dir)
clear_dir(output_dir)
clear_dir(output_temp_dir)

In [ ]:
# prepare filelist
infilenames_list = []
outfilenames_list = []

for i in np.arange(0, len(z_files)):
    zfile = z_files[i]
    zsfile = zsfile
    mslfile = msl_files[i]
    u10file = u10_files[i]
    v10file = v10_files[i]

    infilenames_list.append(f"{zfile};{zsfile};{mslfile};{u10file};{v10file}")
    outfilenames_list.append(f"{output_temp_dir}/detectNode_output_era5_{str(year_sta)}-{str(year_end)}_{str(i+1).rjust(2,'0')}.txt")
    
    write_to_filelist(infilenames_list,f'{input_dir}/detectNode_input_era5_{str(year_sta)}-{str(year_end)}.txt')
    write_to_filelist(outfilenames_list,f'{output_dir}/detectNode_output_era5_{str(year_sta)}-{str(year_end)}.txt')

In [ ]:
# Run TempestExtremes DetectNode
os.environ['TEMPESTEXTREMESDIR']='/home/565/cj0591/tempestextremes/bin'

run_detectNodes(f'{input_dir}/detectNode_input_era5_{str(year_sta)}-{str(year_end)}.txt', # input filelist
                f'{output_dir}/detectNode_output_era5_{str(year_sta)}-{str(year_end)}.txt', # output filelist
                28, # nodes for mpi parallel runnning
                detect_var="msl", # msl used for detecting cyclone centres
                merge_dist=6.0,
                closedcontour_commands="msl,200.0,5.5,0;_DIFF(z(300millibars),z(500millibars)),-58.8,6.5,1.0"
                output_commands="msl,min,0;_VECMAG(u10,v10),max,2.0;zs,min,0",
                timeinterval="6hr",
                lonname="longitude",latname="latitude",
                logdir=f"{base_dir}/log/",
                quiet=True)

In [ ]:
# Run TempestExtremes DetectNode
run_stitchNodes(f"{output_dir}/detectNode_output_era5_{str(year)}.txt",
                f"{txt_dir}/sitchNode_output_era5_{str(year)}.txt", 1,
                output_filefmt="gfdl",
                range_dist=8.0,
                minim_time="54h",
                maxgap_time="24h",
                threshold_condition="wind,>=,10.0,10;lat,<=,50.0,10;lat,>=,-50.0,10;zs,<,150,10",
                quiet=True)

CPU times: user 523 ms, sys: 176 ms, total: 699 ms
Wall time: 2.12 s
